In [71]:
#Setting up the code and installing required packages 
%load_ext autotime
from util import *
from glob import glob
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.geometry import LineString, Polygon
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np
from shapelysmooth import taubin_smooth
pd.set_option("display.max_columns", None)

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


In [72]:
#Navigate to intersects shapefile and separates out the year, finds how many years from 1800 and how many years to 2100
gdf = gpd.read_file(f"Data/Merged Intersects_UniqueID/JackettIsland_Intersects.shp")
gdf["Date"] = pd.to_datetime(gdf.ShorelineI, dayfirst=True, format='mixed')
gdf["Year"] = gdf.Date.dt.year
gdf["YearsSinceBase"] = (gdf.Date - pd.Timestamp(1800, 1, 1)).dt.days / 365.25
gdf["YearsUntilFuture"] = (
    pd.Timestamp(2100, 1, 1) - gdf.Date
    ).dt.days / 365.25
gdf.Date = gdf.Date.astype(str)
gdf["TransectID"] = gdf.Unique_ID.astype(np.int64)
gdf

,ShorelineI,BaselineID,Distance,IntersectX,IntersectY,Uncertaint,Unique_ID,Date,geometry,Year,YearsSinceBase,YearsUntilFuture,TransectID
0,04/19/2017,1.0,-97.80,1.602805e+06,5.444480e+06,2.94,2.010001e+11,2017-04-19,POINT Z (1602805.403 5444479.642 0.000),2017,217.292266,82.702259,201000147441
1,01/13/2000,1.0,-54.62,1.602842e+06,5.444502e+06,5.32,2.010001e+11,2000-01-13,POINT Z (1602842.047 5444502.478 0.000),2000,200.027379,99.967146,201000147441
2,02/19/2010,1.0,-69.27,1.602830e+06,5.444495e+06,2.32,2.010001e+11,2010-02-19,POINT Z (1602829.612 5444494.729 0.000),2010,210.130048,89.864476,201000147441
3,09/13/1985,1.0,-54.69,1.602842e+06,5.444502e+06,4.33,2.010001e+11,1985-09-13,POINT Z (1602841.988 5444502.442 0.000),1985,185.694730,114.299795,201000147441
4,02/04/1967,1.0,-65.04,1.602833e+06,5.444497e+06,5.94,2.010001e+11,1967-04-02,POINT Z (1602833.209 5444496.970 0.000),1967,167.244353,132.750171,201000147441
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2337,02/22/1940,3.0,-29.51,1.603246e+06,5.444123e+06,3.77,2.010002e+11,1940-02-22,POINT Z (1603246.456 5444123.385 0.000),1940,140.136893,159.857632,201000205029
2338,06/25/2013,3.0,-113.28,1.603204e+06,5.444051e+06,2.30,2.010002e+11,2013-06-25,POINT Z (1603203.881 5444051.243 0.000),2013,213.475702,86.518823,201000205029
2339,08/28/2006,3.0,-109.38,1.603206e+06,5.444055e+06,2.32,2.010002e+11,2006-08-28,POINT Z (1603205.860 5444054.597 0.000),2006,206.650240,93.344285,201000205029
2340,01/31/1980,3.0,-115.24,1.603203e+06,5.444050e+06,5.31,2.010002e+11,1980-01-31,POINT Z (1603202.885 5444049.556 0.000),1980,180.076660,119.917864,201000205029


In [3]:
# Filter to just Karekare for testing
#gdf = gdf.cx[1727077.5508:1735465.0761,5897869.0262:5910640.2418]
#gdf

,ShorelineI,BaselineID,Distance,IntersectX,IntersectY,Uncertaint,Unique_ID,Date,geometry,Year,YearsSinceBase,YearsUntilFuture,TransectID
0,01/03/2011,0.0,-37.16,1.730591e+06,5.908710e+06,8.59,1.006327e+11,2011-03-01,POINT Z (1730590.982 5908710.362 0.000),2011,211.156742,88.837782,100632729920
1,08/19/1960,0.0,-51.70,1.730600e+06,5.908699e+06,3.76,1.006327e+11,1960-08-19,POINT Z (1730600.339 5908699.234 0.000),1960,160.626968,139.367556,100632729920
2,01/04/2017,0.0,-32.09,1.730588e+06,5.908714e+06,2.94,1.006327e+11,2017-04-01,POINT Z (1730587.720 5908714.241 0.000),2017,217.242984,82.751540,100632729920
3,01/02/2004,0.0,-48.27,1.730598e+06,5.908702e+06,10.05,1.006327e+11,2004-02-01,POINT Z (1730598.132 5908701.858 0.000),2004,204.079398,95.915127,100632729920
4,01/04/2022,0.0,-23.18,1.730582e+06,5.908721e+06,2.07,1.006327e+11,2022-04-01,POINT Z (1730581.983 5908721.063 0.000),2022,222.242300,77.752225,100632729920
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85101,04/05/2016,2.0,-288.76,1.731087e+06,5.904261e+06,8.84,1.006321e+11,2016-05-04,POINT Z (1731087.039 5904261.021 0.000),2016,216.334018,83.660507,100632146004
85102,04/07/2010,2.0,-284.87,1.731083e+06,5.904261e+06,8.84,1.006321e+11,2010-07-04,POINT Z (1731083.142 5904260.921 0.000),2010,210.499658,89.494867,100632146004
85103,04/30/2024,2.0,-347.31,1.731146e+06,5.904263e+06,8.83,1.006321e+11,2024-04-30,POINT Z (1731145.565 5904262.521 0.000),2024,224.323066,75.671458,100632146004
85104,05/18/2020,2.0,-297.29,1.731096e+06,5.904261e+06,8.84,1.006321e+11,2020-05-18,POINT Z (1731095.558 5904261.239 0.000),2020,220.372348,79.622177,100632146004


In [73]:
def get_transects(intersects):
  p1 = intersects.geometry[intersects.Distance.idxmin()].coords[0]
  p2 = intersects.geometry[intersects.Distance.idxmax()].coords[0]
  azimuth = math.degrees(math.atan2(p1[0]-p2[0], p1[1]-p2[1]))
  if azimuth < 0:
      azimuth += 360
  return pd.Series({"Azimuth": azimuth, "geometry": LineString([p1, p2])})

lines = gdf.groupby("TransectID")[["geometry", "Distance"]].apply(get_transects)
lines.crs = gdf.crs
lines

,Azimuth,geometry
TransectID,,
201000006218,258.690068,"LINESTRING Z (1602185.330 5445755.803 0.000, 1..."
201000007220,258.690068,"LINESTRING Z (1602188.813 5445746.302 0.000, 1..."
201000008225,258.690068,"LINESTRING Z (1602192.240 5445736.789 0.000, 1..."
201000009230,258.690068,"LINESTRING Z (1602195.330 5445727.209 0.000, 1..."
201000010239,258.690068,"LINESTRING Z (1602196.461 5445717.237 0.000, 1..."
...,...,...
201000233653,274.085617,"LINESTRING Z (1603305.544 5443845.332 0.000, 1..."
201000234444,287.241460,"LINESTRING Z (1603305.544 5443845.913 0.000, 1..."
201000234674,274.085617,"LINESTRING Z (1603304.318 5443835.394 0.000, 1..."


In [74]:
lines["dist_to_neighbour"] = lines.distance(lines.shift(-1))
breakpoints = lines.dist_to_neighbour[lines.dist_to_neighbour > 15]
lines["group"] = pd.Series(range(len(breakpoints)), index=breakpoints.index)
lines["group"] = lines.group.bfill().fillna(len(breakpoints)).astype(int)
transect_metadata = lines[["Azimuth", "group"]].to_dict(orient="index")

In [75]:
transect_metadata

{201000006218: {'Azimuth': 258.6900675249549, 'group': 0},
 201000007220: {'Azimuth': 258.69006752786225, 'group': 0},
 201000008225: {'Azimuth': 258.6900681804391, 'group': 0},
 201000009230: {'Azimuth': 258.6900675252886, 'group': 0},
 201000010239: {'Azimuth': 258.6900675268282, 'group': 0},
 201000012112: {'Azimuth': 238.55128489231237, 'group': 1},
 201000013122: {'Azimuth': 238.55128571432044, 'group': 1},
 201000014130: {'Azimuth': 238.55128432263012, 'group': 1},
 201000015137: {'Azimuth': 238.55128432319822, 'group': 1},
 201000016143: {'Azimuth': 238.5512843218965, 'group': 1},
 201000017149: {'Azimuth': 238.55128489223875, 'group': 1},
 201000018155: {'Azimuth': 238.55128628350332, 'group': 1},
 201000019161: {'Azimuth': 238.5512843217919, 'group': 1},
 201000019767: {'Azimuth': 245.07152607707573, 'group': 1},
 201000020767: {'Azimuth': 245.0715260770204, 'group': 1},
 201000021767: {'Azimuth': 245.071526077383, 'group': 1},
 201000022768: {'Azimuth': 245.07152547213863, 'g

In [76]:
#Linear regression is run here. See util.py for the breakdown on linear_models
linear_models = fit(gdf, transect_metadata)
linear_models

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse
0,201000006218,0.141150,-82.918686,0,0.268150,5.411194,38.018848,6.165943
1,201000007220,0.157336,-86.694477,0,0.342575,5.209630,33.215158,5.763259
2,201000008225,0.177292,-91.054082,0,0.410147,5.077690,31.606213,5.621940
3,201000009230,0.201853,-96.181599,0,0.462914,5.061075,33.052558,5.749135
4,201000010239,0.231558,-102.023063,0,0.501750,5.368819,37.228017,6.101477
...,...,...,...,...,...,...,...,...
178,201000233653,-0.142888,-43.510724,6,0.102303,8.237149,125.263260,11.192107
179,201000234444,-0.148141,-43.023745,6,0.103245,8.401957,133.275152,11.544486
180,201000234674,-0.150455,-44.532273,6,0.103355,8.839343,137.308119,11.717855
181,201000235445,-0.163733,-40.195126,6,0.111552,8.961957,149.286022,12.218266


In [8]:
#Only run if rolling average is needed. Otherwise SKIP THIS
#linear_models = fit(gdf, transect_metadata)
#rolled_slopes = linear_models.groupby("group").slope.rolling(10, min_periods=1).mean().dropna().reset_index(level=0)
#linear_models.slope = rolled_slopes.slope
#linear_models.dropna(inplace=True)
#linear_models

time: 361 µs (started: 2024-08-07 16:42:36 +12:00)


In [77]:
#Coordinates of the projected shoreline are plotted here

#Changed coordinate function by making old_x and old_y negative 
def calculate_new_coordinates(old_x, old_y, bearing, distance):
    bearing_radians = math.radians(bearing)
    new_x = old_x + (distance * math.sin(bearing_radians))
    new_y = old_y + (distance * math.cos(bearing_radians))
    point = Point(new_x, new_y)
    assert not point.is_empty
    return point

#Removed other model equations and changed Azimuth addtion from 180 to 360 deg
def predict(
    df: pd.DataFrame,
    linear_models: pd.DataFrame,
    transect_metadata: dict,
):
    """_summary_

    Args:
        df (pd.DataFrame): dataframe with columns: TransectID, Date, Distance, YearsSinceBase
        linear_models (pd.DataFrame): dataframe with columns: TransectID, slope, intercept
        transect_metadata (dict): dict lookup of TransectID to Azimuth & group
        
    Returns:
        pd.DataFrame: resulting prediction points for the year 2100
    """
    results = []
    for i, row in linear_models.iterrows():
        transect_ID = row.TransectID
        transect_df = df[df.TransectID == transect_ID]
        latest_row = transect_df[transect_df.Date == transect_df["Date"].max()].iloc[0]
        future_year = int(row.get("FUTURE_YEAR", FUTURE_YEAR))
        result = row.to_dict()
        result.update({
            "TransectID": transect_ID,
            "BaselineID": latest_row.BaselineID,
            "group": row.group,
            "Year": future_year,
            "ocean_point": calculate_new_coordinates(
                latest_row.geometry.x,
                latest_row.geometry.y,
                transect_metadata[transect_ID]["Azimuth"] + 180,
                500,
            ),
        })
        
        model = "linear"
        slope = row.slope
        intercept = row.intercept

        predicted_distance = slope * (future_year - 1800) + intercept
        distance_difference = latest_row.Distance - predicted_distance
        result[f"{model}_model_point"] = calculate_new_coordinates(
            latest_row.geometry.x,
            latest_row.geometry.y,
            transect_metadata[transect_ID]["Azimuth"],
            distance_difference,
        )
        result[f"{model}_model_predicted_distance"] = predicted_distance
        result[f"{model}_model_distance"] = distance_difference
        results.append(result)
    results = gpd.GeoDataFrame(results)
    return results

In [78]:
#Projection file is created here with the stats and coordinate points in table format
results = predict(gdf, linear_models, transect_metadata)
results

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse,BaselineID,Year,ocean_point,linear_model_point,linear_model_predicted_distance,linear_model_distance
0,2.010000e+11,0.141150,-82.918686,0.0,0.268150,5.411194,38.018848,6.165943,1.0,2100,POINT (1602697.1008380533 5445858.157424874),POINT (1602214.2397982648 5445761.585216907),-40.573573,-7.576427
1,2.010000e+11,0.157336,-86.694477,0.0,0.342575,5.209630,33.215158,5.763259,1.0,2100,POINT (1602700.2877252528 5445848.59676326),POINT (1602217.25974638 5445751.991167502),-39.493818,-7.406182
2,2.010000e+11,0.177292,-91.054082,0.0,0.410147,5.077690,31.606213,5.621940,1.0,2100,POINT (1602703.1610996271 5445838.973393835),POINT (1602220.8168493283 5445742.504549505),-37.866548,-8.103452
3,2.010000e+11,0.201853,-96.181599,0.0,0.462914,5.061075,33.052558,5.749135,1.0,2100,POINT (1602706.0136969849 5445829.345879575),POINT (1602224.9744777898 5445733.13803573),-35.625673,-9.434327
4,2.010000e+11,0.231558,-102.023063,0.0,0.501750,5.368819,37.228017,6.101477,1.0,2100,POINT (1602709.399396347 5445819.824980407),POINT (1602229.9486008768 5445723.93482132),-32.555792,-11.054208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,2.010002e+11,-0.142888,-43.510724,6.0,0.102303,8.237149,125.263260,11.192107,3.0,2100,POINT (1603813.2790788254 5443809.065187531),POINT (1603308.198746566 5443845.14235328),-86.377163,6.367163
179,2.010002e+11,-0.148141,-43.023745,6.0,0.103245,8.401957,133.275152,11.544486,3.0,2100,POINT (1603791.7387139841 5443695.02528319),POINT (1603307.3816384424 5443845.342999348),-87.466125,7.146125
180,2.010002e+11,-0.150455,-44.532273,6.0,0.103355,8.839343,137.308119,11.717855,3.0,2100,POINT (1603809.8078799683 5443799.287652555),POINT (1603304.2072321824 5443835.401983696),-89.668804,6.888804
181,2.010002e+11,-0.163733,-40.195126,6.0,0.111552,8.961957,149.286022,12.218266,3.0,2100,POINT (1603788.1730199924 5443685.661381762),POINT (1603302.6517322832 5443836.34039923),-89.315112,8.365112


In [79]:
#Spatial reference added to the results
results.set_geometry("linear_model_point", inplace=True, crs=2193)
results

,TransectID,slope,intercept,group,r2_score,mae,mse,rmse,BaselineID,Year,ocean_point,linear_model_point,linear_model_predicted_distance,linear_model_distance
0,2.010000e+11,0.141150,-82.918686,0.0,0.268150,5.411194,38.018848,6.165943,1.0,2100,POINT (1602697.1008380533 5445858.157424874),POINT (1602214.240 5445761.585),-40.573573,-7.576427
1,2.010000e+11,0.157336,-86.694477,0.0,0.342575,5.209630,33.215158,5.763259,1.0,2100,POINT (1602700.2877252528 5445848.59676326),POINT (1602217.260 5445751.991),-39.493818,-7.406182
2,2.010000e+11,0.177292,-91.054082,0.0,0.410147,5.077690,31.606213,5.621940,1.0,2100,POINT (1602703.1610996271 5445838.973393835),POINT (1602220.817 5445742.505),-37.866548,-8.103452
3,2.010000e+11,0.201853,-96.181599,0.0,0.462914,5.061075,33.052558,5.749135,1.0,2100,POINT (1602706.0136969849 5445829.345879575),POINT (1602224.974 5445733.138),-35.625673,-9.434327
4,2.010000e+11,0.231558,-102.023063,0.0,0.501750,5.368819,37.228017,6.101477,1.0,2100,POINT (1602709.399396347 5445819.824980407),POINT (1602229.949 5445723.935),-32.555792,-11.054208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,2.010002e+11,-0.142888,-43.510724,6.0,0.102303,8.237149,125.263260,11.192107,3.0,2100,POINT (1603813.2790788254 5443809.065187531),POINT (1603308.199 5443845.142),-86.377163,6.367163
179,2.010002e+11,-0.148141,-43.023745,6.0,0.103245,8.401957,133.275152,11.544486,3.0,2100,POINT (1603791.7387139841 5443695.02528319),POINT (1603307.382 5443845.343),-87.466125,7.146125
180,2.010002e+11,-0.150455,-44.532273,6.0,0.103355,8.839343,137.308119,11.717855,3.0,2100,POINT (1603809.8078799683 5443799.287652555),POINT (1603304.207 5443835.402),-89.668804,6.888804
181,2.010002e+11,-0.163733,-40.195126,6.0,0.111552,8.961957,149.286022,12.218266,3.0,2100,POINT (1603788.1730199924 5443685.661381762),POINT (1603302.652 5443836.340),-89.315112,8.365112


In [80]:
#Line and polygon shapefiles are created here 
def prediction_results_to_line_polygon(results: gpd.GeoDataFrame):
    lines = []
    polygons = []
    for group_name, group_data in results.groupby(["BaselineID", "group"]):
        if len(group_data) > 1:
            # Convert the points to LineString
            line = LineString(list(group_data.geometry))
            lines.append(line)
            # Convert the points to a closed Polygon
            polygon = Polygon(list(group_data.geometry) + list(group_data.ocean_point)[::-1])
            polygons.append(polygon)
    lines = gpd.GeoSeries(lines, crs=2193)
    polygons = gpd.GeoSeries(polygons, crs=2193)
    return lines, polygons
lines, poly = prediction_results_to_line_polygon(results)

In [82]:
m = lines.explore()
lines.apply(lambda line: taubin_smooth(line, steps=500)).to_file("Projections/WaihekeIsland_output_line_smoothed.shp")
lines.apply(lambda line: taubin_smooth(line, steps=500)).explore(m=m, color="red")

In [15]:
#Saving line and polygon projection file to Z drive. Change file location accordingly  
lines, poly = prediction_results_to_line_polygon(results)
lines.to_file("Z:\Lalita\RNC Cont\......\BigBay_projection_output_lines.shp")
poly.to_file("Z:\Lalita\RNC Cont\.......\BigBay_projection_output_polygon.shp")

DriverIOError: Failed to create file Z:\Lalita\RNC Cont\....../BigBay_projection_output_lines.shp: No such file or directory

time: 859 ms (started: 2024-08-07 16:42:39 +12:00)


In [70]:
lines, poly = prediction_results_to_line_polygon(results)
lines.to_file("Projections/WaihekeIsland_projection_output_line.shp")
poly.to_file("Projections/WaihekeIsland_projection_output_polygon.shp")

In [ ]:
#Saving line and polygon projection file to folder in VS Code. Change file location accordingly
lines, poly = prediction_results_to_line_polygon(results)
lines.to_file("Projections\NorthIsland_projection_output_lines.shp")
poly.to_file("Projections\NorthIsland_projection_output_polygon.shp")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 11-12: malformed \N character escape (3394349576.py, line 3)

In [ ]:
#Quick visualisation of projected polygon and historic shorelines 
m = poly.explore(tiles="Esri.WorldImagery")
gpd.GeoDataFrame(results.drop(columns=["ocean_point", "linear_model_point"]), geometry=results.linear_model_point).explore(m=m)
gdf.explore("Year", legend=True, m=m)